![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/HuggingFace_in_Spark_NLP_WhisperForASR.ipynb)

## Import WhisperForASR models from HuggingFace 🤗  into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- This feature is only in `Spark NLP 5.1.0` and after. So please make sure you have upgraded to the latest Spark NLP release
- The Whisper model was introduced in `Spark NLP 5.1.0 and requires Spark versions 3.4.0 and up.`
- Official models are supported, but not all custom models may work.

## Export and Save HuggingFace model

- Let's install `HuggingFace` and `TensorFlow`. You don't need `TensorFlow` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock TensorFlow on `2.11.0` version and Transformers on `4.32.0`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

In [ ]:
!pip install -q transformers==4.39.3 tensorflow==2.11.0 numpy<2 torch

- HuggingFace comes with a native `saved_model` feature inside `save_pretrained` function for TensorFlow based models. We will use that to save it as TF `SavedModel`.
- We'll use the [whisper-tiny](https://huggingface.co/openai/whisper-tiny) model from HuggingFace as an example
- In addition to `TFWhisperForASRModel` we also need to save the `WhisperProcessor`. This is the same for every model, these are assets needed for preprocessing inside Spark NLP.

In [ ]:
MODEL_NAME = "openai/whisper-tiny"
EXPORT_PATH: LiteralString = f"exported_tf/{MODEL_NAME}"
assets_folder = f"{EXPORT_PATH}/assets"

Exporting this model involves several steps. We need to

1. separate the audio encoder and token decoder and their cache tensors
3. create a wrapper to create the right model signatures
4. export the preprocessor to the `assets` folder

Don't worry if this next step seems overwhelming. Once you run the next cell everything should be exported to the right place!

In [3]:
import tensorflow as tf
from tensorflow.python.framework.convert_to_constants import (
    convert_variables_to_constants_v2,
)
from transformers import TFWhisperForConditionalGeneration, WhisperProcessor

encoder_signature = [
    {
        "input_features": tf.TensorSpec(
            (None, None, None), tf.float32, name="input_features"
        )
    }
]

decoder_init_signature = [
    {
        "decoder_input_ids_init": tf.TensorSpec(
            (None, None), tf.int32, name="decoder_input_ids_init"
        ),
        "encoder_state_init": tf.TensorSpec(
            (None, None, None), tf.float32, name="encoder_state_init"
        ),
    }
]

decoder_cache_signature = [
    {
        "decoder_input_ids": tf.TensorSpec(
            (None, None), tf.int32, name="decoder_input_ids"
        ),
        "encoder_state": tf.TensorSpec(
            (None, None, None), tf.float32, name="encoder_state"
        ),
        "decoder_past_key_values": tf.TensorSpec(
            (
                4,
                2,
                None,
                None,
                None,
                64,
            ),  # (layers, key-values, batch_size, num_heads, decoder_sequence_length, embed_size_per_head)
            tf.float32,
            name="decoder_past_key_values",
        ),
        "encoder_past_key_values": tf.TensorSpec(
            (
                4,
                2,
                None,
                None,
                None,
                64,
            ),  # (layers, key-values, batch_size, num_heads, encoder_sequence_length, embed_size_per_head)
            tf.float32,
            name="encoder_past_key_values",
        ),
    }
]


class WhisperExport(TFWhisperForConditionalGeneration):
    def to_logits(self, decoder_last_hidden_state):
        # Decoder and encoder embeddings are tied, convert to logits
        lm_logits = tf.matmul(
            decoder_last_hidden_state,
            self.get_output_embeddings().weights,
            transpose_b=True,
        )
        return lm_logits

    def convert_cache_tensors_to_tuple(
        self, decoder_cache_tensors, encoder_cache_tensors
    ):
        return tuple(
            [
                tuple(
                    [decoder_cache_tensors[i, j] for j in range(2)]
                    + [encoder_cache_tensors[i, j] for j in range(2)]
                )
                for i in range(self.config.decoder_layers)
            ]
        )

    @staticmethod
    def make_cache_tensors(past_key_values):
        return tf.stack(
            [[k for k in tensors[0:2]] for tensors in past_key_values]
        ), tf.stack([[k for k in tensors[2:4]] for tensors in past_key_values])

    def make_encoder_serving(self):
        @tf.function(input_signature=encoder_signature)
        def encoder_serving(inputs):
            return {"last_hidden_state": self.get_encoder()(inputs).last_hidden_state}

        return convert_variables_to_constants_v2(
            encoder_serving.get_concrete_function()
        )

    def make_decoder_init_serving(self):
        @tf.function(input_signature=decoder_init_signature)
        def decoder_init_serving(inputs):
            outputs = self.get_decoder()(
                inputs["decoder_input_ids_init"],
                encoder_hidden_states=inputs["encoder_state_init"],
            )

            lm_logits = self.to_logits(outputs[0])
            decoder_past_key_values, encoder_past_key_values = self.make_cache_tensors(
                outputs.past_key_values
            )
            return {
                "logits_init": lm_logits,
                "decoder_past_key_values": decoder_past_key_values,
                "encoder_past_key_values": encoder_past_key_values,
            }

        return convert_variables_to_constants_v2(
            decoder_init_serving.get_concrete_function()
        )

    def make_decoder_serving(self):
        @tf.function(input_signature=decoder_cache_signature)
        def decoder_serving(inputs):
            past_key_values_in = self.convert_cache_tensors_to_tuple(
                inputs["decoder_past_key_values"], inputs["encoder_past_key_values"]
            )
            outputs = self.get_decoder()(
                inputs["decoder_input_ids"],
                encoder_hidden_states=inputs["encoder_state"],
                past_key_values=past_key_values_in,
                use_cache=True,
            )

            lm_logits = self.to_logits(outputs[0])
            (decoder_past_key_values, _) = self.make_cache_tensors(
                outputs.past_key_values
            )
            return {
                "logits": lm_logits,
                "decoder_past_key_values": decoder_past_key_values,
            }

        return convert_variables_to_constants_v2(
            decoder_serving.get_concrete_function()
        )


class ModelWrapping(tf.Module):
    """
    Wrapper class for specifying signatures
    """

    @tf.function(input_signature=encoder_signature)
    def encoder_serving(self, inputs):
        return {
            "last_hidden_state": self.__encoder_serving_f(
                input_features=inputs["input_features"],
            )[0]
        }

    @tf.function(input_signature=decoder_init_signature)
    def decoder_init_serving(self, inputs):
        decoder_output = self.__decoder_init_serving_f(
            decoder_input_ids_init=inputs["decoder_input_ids_init"],
            encoder_state_init=inputs["encoder_state_init"],
        )
        return {
            "logits_init": decoder_output[2],
            "decoder_past_key_values_init": decoder_output[0],
            "encoder_past_key_values_init": decoder_output[1],
        }

    @tf.function(input_signature=decoder_cache_signature)
    def decoder_cache_serving(self, inputs):
        decoder_output = self.__decoder_cache_serving_f(
            decoder_input_ids=inputs["decoder_input_ids"],
            encoder_state=inputs["encoder_state"],
            decoder_past_key_values=inputs["decoder_past_key_values"],
            encoder_past_key_values=inputs["encoder_past_key_values"],
        )
        return {
            "logits": decoder_output[1],
            "decoder_past_key_values_out": decoder_output[0],
        }

    def export(self, export_path):
        signatures = {
            "encoder": self.encoder_serving,
            "decoder_init": self.decoder_init_serving,
            "decoder_with_past": self.decoder_cache_serving,
        }

        tf.saved_model.save(self, export_path, signatures=signatures)

    def __init__(
        self, encoder_serving_f, decoder_init_serving_f, decoder_cache_serving_f
    ):
        self.__encoder_serving_f = encoder_serving_f
        self.__decoder_init_serving_f = decoder_init_serving_f
        self.__decoder_cache_serving_f = decoder_cache_serving_f


whisper_model = WhisperExport.from_pretrained(MODEL_NAME, from_pt=True)

model_wrapping = ModelWrapping(
    whisper_model.make_encoder_serving(),
    whisper_model.make_decoder_init_serving(),
    whisper_model.make_decoder_serving(),
)

model_wrapping.export(EXPORT_PATH)
whisper_model.config.save_pretrained(assets_folder)

processor = WhisperProcessor.from_pretrained(MODEL_NAME)
processor.save_pretrained(assets_folder)

/home/ducha/spark-nlp/spark-nlp-clean/venv_tf/lib/python3.10/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


All PyTorch model weights were used when initializing WhisperExport.

All the weights of WhisperExport were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use WhisperExport for predictions without further training.
2025-08-10 11:05:39.873963: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-08-10 11:05:39.877623: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2025-08-10 11:05:39.877764: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2025-08-10 11:05:39.878021: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


2025-08-10 11:05:42.835438: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-08-10 11:05:42.836473: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2025-08-10 11:05:42.836553: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2025-08-10 11:05:42.836818: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-08-10 11:05:42.837689: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gp

Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
INFO:tensorflow:Assets written to: exported_tf/openai/whisper-tiny/assets


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50358, 50359, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}
/home/ducha/spark-nlp/spark-nlp-clean/venv_tf/lib/python3.10/site-packages/huggingface_hub/f

[]

Let's have a look inside these two directories and see what we are dealing with:

In [4]:
!ls -l {EXPORT_PATH}

total 259580
drwxr-xr-x 2 ducha ducha      4096 Aug 10 11:05 assets
-rw-rw-r-- 1 ducha ducha        56 Aug 10 11:05 fingerprint.pb
-rw-rw-r-- 1 ducha ducha 265791871 Aug 10 11:05 saved_model.pb
drwxr-xr-x 2 ducha ducha      4096 Aug 10 11:05 variables


In [5]:
!ls -l {EXPORT_PATH}/assets

total 1880
-rw-rw-r-- 1 ducha ducha   34604 Aug 10 11:05 added_tokens.json
-rw-rw-r-- 1 ducha ducha    2271 Aug 10 11:05 config.json
-rw-rw-r-- 1 ducha ducha  493869 Aug 10 11:05 merges.txt
-rw-rw-r-- 1 ducha ducha   52666 Aug 10 11:05 normalizer.json
-rw-rw-r-- 1 ducha ducha     339 Aug 10 11:05 preprocessor_config.json
-rw-rw-r-- 1 ducha ducha    2194 Aug 10 11:05 special_tokens_map.json
-rw-rw-r-- 1 ducha ducha  282683 Aug 10 11:05 tokenizer_config.json
-rw-rw-r-- 1 ducha ducha 1036584 Aug 10 11:05 vocab.json


## Import and Save WhisperForASR in Spark NLP


- Let's install and setup Spark NLP in Google Colab
- This part is pretty easy via our simple script
- Additionally, we will need to upgrade Spark to version 3.4.1

In [6]:
! wget -q http://setup.johnsnowlabs.com/colab.sh -O - | bash
! pip install -U pyspark==3.4.1

Installing PySpark 3.2.3 and Spark NLP 5.3.3
setup Colab for PySpark 3.2.3 and Spark NLP 5.3.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.5/281.5 MB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.4/568.4 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 20.6 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285388 sha256=c379cadeb03663e854d5f5f547cac2ca264ac796ccaa2d71c5d48e7c56bc3084
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.5
    Uninstalling py4j-0.10.9.5:
      Suc

Let's start Spark with Spark NLP included via our simple `start()` function

In [1]:
import sparknlp

# let's start Spark with Spark NLP
spark = sparknlp.start()

25/08/10 11:11:14 WARN Utils: Your hostname, pop-os resolves to a loopback address: 127.0.1.1; using 192.168.0.30 instead (on interface enp3s0)
25/08/10 11:11:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/ducha/Workspace/scala/spark-nlp-clean/venv_tf/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ducha/.ivy2/cache
The jars for the packages stored in: /home/ducha/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b437ff1e-3d3e-4d08-aa46-5d5b05a125d9;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;6.1.1 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-s3;1.12.500 in central
	found com.amazonaws#aws-java-sdk-kms;1.12.500 in central
	found com.amazonaws#aws-java-sdk-core;1.12.500 in central
	found commons-logging#commons-logging;1.1.3 in central
	found commons-codec#commons-codec;1.15 in central
	found org.apache.httpcomponents#httpclient;4.5.13 in central
	found org.apache.httpcomponents#httpcore;4.4.13 in central
	found software.amazon.ion#ion-java;1.0.2 in central
	found joda-time#joda-time;2.8.1 in central
	found com.amazonaws#jmespath-java;1.12.500 in central


- Let's use `loadSavedModel` functon in `WhisperForASR` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `WhisperForASR` in runtime, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.

In [4]:
from sparknlp.annotator import *

MODEL_NAME = "openai/whisper-tiny"
EXPORT_PATH = f"exported_tf/{MODEL_NAME}"

whisper = (
    WhisperForASR.loadSavedModel(f"{EXPORT_PATH}", spark)
    .setInputCols("audio_assembler")
    .setOutputCol("text")
)

2025-08-10 11:12:12.288407: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /home/ducha/Workspace/scala/spark-nlp-clean/examples/python/transformers/exported_tf/openai/whisper-tiny
2025-08-10 11:12:12.390822: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:107] Reading meta graph with tags { serve }
2025-08-10 11:12:12.390846: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:148] Reading SavedModel debug info (if present) from: /home/ducha/Workspace/scala/spark-nlp-clean/examples/python/transformers/exported_tf/openai/whisper-tiny
2025-08-10 11:12:12.391699: I external/org_tensorflow/tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-10 11:12:12.462726:

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [9]:
whisper.write().overwrite().save("./{}_spark_nlp".format(MODEL_NAME))

Let's clean up stuff we don't need anymore

In [10]:
!rm -rf {EXPORT_PATH}

Awesome 😎  !

This is your WhisperForASR model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [11]:
! ls -l {MODEL_NAME}_spark_nlp

total 259600
drwxr-xr-x 7 root root      4096 Apr 12 18:17 fields
drwxr-xr-x 2 root root      4096 Apr 12 18:17 metadata
-rw-r--r-- 1 root root 265819147 Apr 12 18:17 whisper_ctc_tensorflow


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny WhisperForASR model 😊

In [5]:
!wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/src/test/resources/audio/txt/librispeech_asr_0.txt

--2025-08-10 11:12:53--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/src/test/resources/audio/txt/librispeech_asr_0.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2199992 (2.1M) [text/plain]
Saving to: ‘librispeech_asr_0.txt’

librispeech_asr_0.t 100%[===================>]   2.10M  --.-KB/s    in 0.1s    

2025-08-10 11:12:53 (14.7 MB/s) - ‘librispeech_asr_0.txt’ saved [2199992/2199992]



In [ ]:
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline


spark = sparknlp.start()

MODEL_NAME = "openai/whisper-tiny"

audioAssembler = (
    AudioAssembler().setInputCol("audio_content").setOutputCol("audio_assembler")
)

# speechToText = WhisperForASR.load(f"{MODEL_NAME}_spark_nlp")

pipeline = Pipeline().setStages([audioAssembler, whisper])

audio_path = "librispeech_asr_0.txt"

# Need to load as 16k hz sampled audio data
with open(audio_path) as file:
    raw_floats = [float(data) for data in file.read().strip().split("\n")]

processedAudioFloats = spark.createDataFrame([[raw_floats]]).toDF("audio_content")

result = pipeline.fit(processedAudioFloats).transform(processedAudioFloats)
result.select("text.result").show(truncate=False)

+------------------------------------------------------------------------------------------+
|result                                                                                    |
+------------------------------------------------------------------------------------------+
|[ Mr. Quilter is the apostle of the middle classes and we are glad to welcome his gospel.]|
+------------------------------------------------------------------------------------------+



That's it! You can now go wild and use hundreds of WhisperForASR models from HuggingFace 🤗 in Spark NLP 🚀
